In [ ]:
#행정구역별 의료기관 현황 분석해보기
#블록맵을 시각화
#필요한 데이터 준비물
#1) 공공보건의료 기관 현황.csv
#2) 행정 구역별 시군구 인구수 엑셀 파일

##작업
##1. 행정구역별 주소 수정
##2. 행정구역별 의료기관 수 집계
##3. 행정구역별 인구수 정리
##4. 테이블 정리, 내부(공통된 행만 뽑아서), 외부(모든 행을 다 합치기)

###블록맵 : 구역의 경계선 단순화 작업하고 -> 블록(사각형모양으로) 그려서 지도에 표기
###히트맵으로 온도로 시각화하는 것과 거의 비슷

In [ ]:
#데이터 수집.
#공공데이터 -> 공공보건 의료기관 데이터 csv 다운
#보건복지부_공공보건 의료기관 현황 엑셀 파일

In [4]:
import pandas as pd 
pd.set_option("mode.chained_assignment",None)
import numpy as np 

data = pd.read_csv("./공공의료기관현황.csv",index_col=0, encoding="cp949", engine="python")
data


병원 명  설립형태                     근거 법령     관계 행정기관 관계 공공단체  \
연번                                                                     
1       강원도 재활병원   시도립         강원도재활병원설치및운영에관한조례         강원도    해당없음   
2      강원도 삼척의료원  특수법인     지방의료원의 설립 및 운영에 관한 법률  보건복지부(강원도)   지방의료원   
3      강원도 영월의료원  특수법인     지방의료원의 설립 및 운영에 관한 법률  보건복지부(강원도)   지방의료원   
4      강원도 원주의료원  특수법인     지방의료원의 설립 및 운영에 관한 법률  보건복지부(강원도)   지방의료원   
5      강원도 강릉의료원  특수법인     지방의료원의 설립 및 운영에 관한 법률  보건복지부(강원도)   지방의료원   
..           ...   ...                       ...         ...     ...   
216    청풍호노인사랑병원  시군구립  제천시 노인전문병원 설치및 운영에 관한 조례        충청북도    해당없음   
217   충주시 노인전문병원  시군구립    충주시노인전문병원설치 및 운영에관한 조례        충청북도    해당없음   
218   충청북도 충주의료원  특수법인     지방의료원의 설립 및 운영에 관한 법률        충청북도    해당없음   
219   영동군립노인전문병원  시군구립    영동군립노인전문병원설치 및    운영조례        충청북도    해당없음   
220  단양군노인요양전문병원  시군구립    단양군립노인전문병원설치 및    운영조례        충청북도    해당없음   

     심평원 요양기관번호  종별구분  병상수 소재지 우편번호                            주소  \
연번                                                                  
1      32200641    병원  165    24227       강원도 춘천시 충열로 142번길 24-16   
2      32100060  종합병원  152    25920              강원도 삼척시 오십천로 418   
3      32100078  종합병원  214    26234           강원도 영월군 영월읍 중앙1로 59   
4      32100086  종합병원  237    26448         강원도 원주시 서원대로 387(개운동)   
5      32100159  종합병원  137    25535   강원도 강릉시 경강로 2007(남문동 164-1)   
..          ...   ...  ...      ...                           ...   
216    33280576  요양병원  187    27207        충청북도 제천시 금성면 청풍호로 1147   
217    33280088  요양병원  300    27442      충청북도 충주시 동량면 대전리 1686-28   
218    33100063  종합병원  292    27468  충청북도 충주시 안림로 239-50(안림동 134)   
219    33280681  요양병원  120    29131          충청북도 영동군 영동읍 대학로 290   
220    33280207  요양병원  105    27010          충청북도 단양군 단양읍 별곡6길 10   

                        홈페이지          대표전화           FAX   비고  
연번                                                             
1              www.grh.or.kr  033-248-7700  033-248-7723  NaN  
2          http://ksmc.or.kr  033-572-1141  033-573-8424  NaN  
3    http://www.youngwol.org  033-370-9117  033-370-9137  NaN  
4             www.kwmc.or.kr  033-760-4500  033-761-5121  NaN  
5      http://www.gnmc.or.kr  033-646-6910  033-610-1415  NaN  
..                       ...           ...           ...  ...  
216         www.cplove.co.kr  043-646-0808  043-648-5827  NaN  
217          www.hyolife.com  043-855-9393  043-855-8866  NaN  
218          www.cjmct.or.kr  043-871-0114  043-843-4231  NaN  
219          cmisilveryd.com  043-744-9633  043-742-8275  NaN  
220                       없음  043-423-8844  043-423-8874  NaN  

[227 rows x 14 columns]

In [8]:
#공공 의료기관 주소 데이터 정리 작업.
addr = pd.DataFrame(data["주소"].apply(lambda v : v.split()[:2]).tolist(),columns=["시도","군구"])

In [15]:
#해당 주소에서 1차 지역명, 2차 지역명으로 추출 후 중복제거
addr["시도"].unique()

#창원시 ->경상남도에 포함, 시도 컬럼에 있으면 안됨, 2차 지역명 군구에 있어야 함
#경산시 ->경상북도에 포함
#천안시 ->충청남도

addr[addr["시도"]=="창원시"]
addr.iloc[27] = ["경상남도","창원시"]
addr.iloc[31] = ["경상남도","창원시"]
print(addr.iloc[27])
print(addr.iloc[31])

addr[addr["시도"]=="경산시"]
addr.iloc[47] = ["경상북도","경산시"]
addr.iloc[47]

addr[addr["시도"]=="천안시"]
addr.iloc[209] = ["충청남도","천안시"]
addr.iloc[210] = ["충청남도","천안시"]
print(addr.iloc[209])
print(addr.iloc[210])
# addr.iloc[47]

addr["시도"].unique()


시도    경상남도
군구     창원시
Name: 27, dtype: object
시도    경상남도
군구     창원시
Name: 31, dtype: object
시도    충청남도
군구     천안시
Name: 209, dtype: object
시도    충청남도
군구     천안시
Name: 210, dtype: object


array(['강원도', '경기도', '경기', '경남', '경상남도', '경상북도', '경북', '인천광역시', '대구광역시',
       '전라남도', '대전광역시', '광주광역시', '제주특별자치도', '부산광역시', '전라북도', '충북',
       '서울특별시', '서울시', '부산특별시', '대전시', '충남', '전남', '충청남도', '울산광역시', '전북',
       '충청북도'], dtype=object)

In [16]:
#1차 지역명 통일하기
# 강원도 ->강원특별자치도
# 경기 ->경기도
# 경남 ->경상남도
# 경북 ->경상북도
# 충북 ->충청북도
# 서울시 ->서울특별시
# 부산특별시 ->부산광역시
# 기타 충남,전남,전북.. 수정
# 대전시 ->대전광역시

addr_alias = {
  "경기" : "경기도",
  "강원도" : "강원특별자치도",
  "경남" : "경상남도",
  "경북" : "경상북도",
  "충북" : "충청북도",
  "서울시" : "서울특별시",
  "부산특별시" : "부산광역시",
  "충남" : "충청남도",
  "전남" : "전라남도",
  "전북" : "전라북도",
  "대전시" : "대전광역시"
}

addr["시도"] = addr["시도"].apply(lambda v:addr_alias.get(v,v))
addr["시도"].unique()


array(['강원특별자치도', '경기도', '경상남도', '경상북도', '인천광역시', '대구광역시', '전라남도',
       '대전광역시', '광주광역시', '제주특별자치도', '부산광역시', '전라북도', '충청북도', '서울특별시',
       '충청남도', '울산광역시'], dtype=object)

In [21]:
#2차 지역명 데이터 확인
addr["군구"].unique()

#"아란13길" : 제주시로 변경하기
addr[addr["군구"]== "아란13길"]
addr.iloc[75] = ["제주특별자치도","제주시"]

#바뀐거 확인
addr["군구"].unique()
addr.iloc[75]

시도    제주특별자치도
군구        제주시
Name: 75, dtype: object

In [22]:
#행정구역별 의료기관 수 확인
#새 컬럼 추가(시도군구)
addr["시도군구"]=addr.apply(lambda r : r["시도"]+ " " +r["군구"], axis=1)
addr.head()

시도   군구         시도군구
0  강원특별자치도  춘천시  강원특별자치도 춘천시
1  강원특별자치도  삼척시  강원특별자치도 삼척시
2  강원특별자치도  영월군  강원특별자치도 영월군
3  강원특별자치도  원주시  강원특별자치도 원주시
4  강원특별자치도  강릉시  강원특별자치도 강릉시